# Cyclistic Bike-Share Case Study

### Import necessary packages and master dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [3]:
# Import dataset 
bikes = pd.read_csv('/Users/adamteske/Desktop/Bikes_dbs/bikes_maybe2.csv')

### Create new columns for starting datetime, ride length, and day of the week.

In [4]:
bikes['start_date'] = pd.to_datetime(bikes['started_at'],format='%m/%d/%y %H:%M')
bikes['end_date'] = pd.to_datetime(bikes['ended_at'],format='%m/%d/%y %H:%M')
bikes['ride_length'] = bikes.end_date - bikes.start_date
bikes['weekday'] = bikes.start_date.dt.dayofweek
bikes.sort_values('start_date',inplace=True)

### Filter out rides that were longer than 24 hours or shorter than 0 hours (negative).

In [6]:
df = bikes[bikes['ride_length']<='1 days 00:00:00']
df = df[df['ride_length']>'0 days 00:00:00']
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,end_date,ride_length,weekday
3372881,75FA47BE49CA9CE1,electric_bike,6/1/22 0:00,6/1/22 0:19,Elston Ave & Wabansia Ave,TA1309000032,NaN,NaN,41.912975,-87.664183,41.950000,-87.670000,member,2022-06-01 00:00:00,2022-06-01 00:19:00,0 days 00:19:00,2
6129631,D6750D9CD9900AF6,electric_bike,6/1/22 0:00,6/1/22 0:10,Wells St & Elm St,KA1504000135,Wells St & Evergreen Ave,TA1308000049,41.903174,-87.634702,41.906724,-87.634830,casual,2022-06-01 00:00:00,2022-06-01 00:10:00,0 days 00:10:00,2
280196,09D24A3C16552534,electric_bike,6/1/22 0:00,6/1/22 0:13,NaN,NaN,NaN,NaN,41.870000,-87.680000,41.850000,-87.710000,casual,2022-06-01 00:00:00,2022-06-01 00:13:00,0 days 00:13:00,2
5517425,C10099D850C7FC17,classic_bike,6/1/22 0:00,6/1/22 0:02,Broadway & Argyle St,13108,Broadway & Berwyn Ave,13109,41.973815,-87.659660,41.978353,-87.659753,member,2022-06-01 00:00:00,2022-06-01 00:02:00,0 days 00:02:00,2
2308754,50C6E795BB1450FD,electric_bike,6/1/22 0:01,6/1/22 0:38,NaN,NaN,NaN,NaN,41.910000,-87.640000,41.950000,-87.650000,casual,2022-06-01 00:01:00,2022-06-01 00:38:00,0 days 00:37:00,2


### Number of trips per rider type

In [12]:
df.groupby('member_casual').count()["ride_id"]

member_casual
casual    2897662
member    4302052
Name: ride_id, dtype: int64

### Number of trips per bike and rider type


In [6]:
df.groupby(['rideable_type','member_casual']).count()["ride_id"]

rideable_type  member_casual
classic_bike   casual           1104551
               member           2125290
docked_bike    casual            186887
electric_bike  casual           1606224
               member           2176762
Name: ride_id, dtype: int64

### Average trip length per rider type

In [7]:
df.groupby(['rideable_type','member_casual'])['ride_length'].mean()

rideable_type  member_casual
classic_bike   casual          0 days 00:24:43.537020925
               member          0 days 00:13:25.754951089
docked_bike    casual          0 days 00:51:46.659425214
electric_bike  casual          0 days 00:15:40.427038818
               member          0 days 00:11:36.498900660
Name: ride_length, dtype: timedelta64[ns]

### Average trip length per rider type by weekday

In [9]:
df.groupby(['weekday','member_casual'])['ride_length'].mean()
# Note that 0 is Monday, 1 is Tuesday, 2 is Wednesday, etc.

weekday  member_casual
0        casual          0 days 00:21:19.304013164
         member          0 days 00:11:56.169180595
1        casual          0 days 00:19:23.252638973
         member          0 days 00:11:58.121585649
2        casual          0 days 00:18:26.557659603
         member          0 days 00:11:53.630646100
3        casual          0 days 00:19:04.776679852
         member          0 days 00:12:03.342338976
4        casual          0 days 00:20:41.955187377
         member          0 days 00:12:22.860989759
5        casual          0 days 00:24:03.123892068
         member          0 days 00:13:57.435757215
6        casual          0 days 00:24:40.051731049
         member          0 days 00:13:53.732024464
Name: ride_length, dtype: timedelta64[ns]

### Total trips and average trip length per rider type by month

In [7]:
df_month = df
df_month['month'] = df_month['start_date'].dt.month
df_month = df_month[df_month['start_date']<='6/1/23 0:00']
df_month.groupby(['member_casual','month'])['ride_id'].count()

member_casual  month
casual         1         39221
               2         42187
               3         60886
               4        144112
               5        229785
               6        364032
               7        400472
               8        353794
               9        292650
               10       205931
               11        99204
               12        44121
member         1        147151
               2        144367
               3        191869
               4        272902
               5        363718
               6        395090
               7        411874
               8        421223
               9        399355
               10       344736
               11       233760
               12       134673
Name: ride_id, dtype: int64

### Most Popular Start and End Stations per rider type

In [10]:
df_station = df.groupby(['member_casual','start_station_name'],as_index=False)['ride_id'].count()
df_station.sort_values(by='ride_id',ascending=False,inplace=True)
df_station.head(20)

,member_casual,start_station_name,ride_id
1707,casual,Streeter Dr & Grand Ave,70501
339,casual,DuSable Lake Shore Dr & Monroe St,40953
764,casual,Michigan Ave & Oak St,32754
340,casual,DuSable Lake Shore Dr & North Blvd,30932
2424,member,Kingsbury St & Kinzie St,30735
771,casual,Millennium Park,30648
2080,member,Clark St & Elm St,28681
2107,member,Clinton St & Washington Blvd,26919
3474,member,Wells St & Concord Ln,26781
1622,casual,Shedd Aquarium,24817


In [22]:
df[df['start_station_name']=='Wells St & Elm St'].head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,end_date,ride_length,weekday,month
6129631,D6750D9CD9900AF6,electric_bike,6/1/22 0:00,6/1/22 0:10,Wells St & Elm St,KA1504000135,Wells St & Evergreen Ave,TA1308000049,41.903174,-87.634702,41.906724,-87.634830,casual,2022-06-01 00:00:00,2022-06-01 00:10:00,0 days 00:10:00,2,6
1534358,35B86EEF1797749F,classic_bike,6/1/22 5:09,6/1/22 5:21,Wells St & Elm St,KA1504000135,Clinton St & Madison St,TA1305000032,41.903222,-87.634324,41.882242,-87.641066,casual,2022-06-01 05:09:00,2022-06-01 05:21:00,0 days 00:12:00,2,6
4804641,A8131DD2BD415BB3,classic_bike,6/1/22 5:49,6/1/22 5:54,Wells St & Elm St,KA1504000135,State St & Chicago Ave,21544,41.903222,-87.634324,41.896617,-87.628579,member,2022-06-01 05:49:00,2022-06-01 05:54:00,0 days 00:05:00,2,6
280371,09D3E5F037069223,classic_bike,6/1/22 5:57,6/1/22 6:00,Wells St & Elm St,KA1504000135,Franklin St & Chicago Ave,13017,41.903222,-87.634324,41.896747,-87.635668,member,2022-06-01 05:57:00,2022-06-01 06:00:00,0 days 00:03:00,2,6
6666922,E945171C5286DB2E,classic_bike,6/1/22 7:20,6/1/22 7:28,Wells St & Elm St,KA1504000135,Wells St & Hubbard St,TA1307000151,41.903222,-87.634324,41.889906,-87.634266,member,2022-06-01 07:20:00,2022-06-01 07:28:00,0 days 00:08:00,2,6
